In [2]:
# Gerekli Kütüphanelerin İçe Aktarılması
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hiperparametre Arama Alanı
param_space = {
    "learning_rate": (0.0001, 0.1),
    "batch_size": [16, 32, 64, 128, 256],
    "hidden_layers": (1, 5),
    "neurons_per_layer": (16, 128),
    "activation": ["relu", "tanh", "sigmoid"],
    "optimizer": ["adam", "sgd", "RMSProp"]
}

# Turnuva Seçimi
def tournament_selection(population, fitnesses, k=5):
    selected = random.sample(range(len(population)), k)
    best_idx = max(selected, key=lambda idx: fitnesses[idx])
    return population[best_idx]

# Fitness Fonksiyonu
def fitness_function(individual, dataset, input_shape, num_classes):
    train_data, val_data = dataset

    # Modeli oluştur
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Flatten())
    for _ in range(individual["hidden_layers"]):
        model.add(Dense(individual["neurons_per_layer"], activation=individual["activation"]))
    model.add(Dense(num_classes, activation="softmax"))

    # Modeli derle
    model.compile(
        optimizer=individual["optimizer"],
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Modeli eğit
    model.fit(train_data, epochs=1, batch_size=individual["batch_size"], verbose=0)
    _, accuracy = model.evaluate(val_data, verbose=0)
    return accuracy

# Genetik Algoritma
def genetic_algorithm(param_space, dataset, pop_size=10, generations=5, mutation_prob=0.05, crossover_prob=0.3):
    input_shape = (100, 100, 3)
    num_classes = 141

    # Popülasyonu oluştur
    population = [
        {
            "learning_rate": random.uniform(*param_space["learning_rate"]),
            "batch_size": random.choice(param_space["batch_size"]),
            "hidden_layers": random.randint(*param_space["hidden_layers"]),
            "neurons_per_layer": random.randint(*param_space["neurons_per_layer"]),
            "activation": random.choice(param_space["activation"]),
            "optimizer": random.choice(param_space["optimizer"])
        }
        for _ in range(pop_size)
    ]

    best_individual = None
    best_fitness = -float("inf")
    all_generations_fitness = []

    for generation in range(generations):
        fitnesses = [fitness_function(ind, dataset, input_shape, num_classes) for ind in population]

        # En iyi bireyi güncelle
        if max(fitnesses) > best_fitness:
            best_fitness = max(fitnesses)
            best_individual = population[np.argmax(fitnesses)]

        all_generations_fitness.append(best_fitness)  # Bu jenerasyonun en iyi fitness'ını kaydet

        # Yeni popülasyonu oluştur
        new_population = []
        for _ in range(pop_size // 2):
            # Turnuva seçimi
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)

            # Tek Noktalı Çaprazlama
            if random.random() < crossover_prob:
                cut_point = random.randint(1, len(parent1) - 1)
                child1 = {**dict(list(parent1.items())[:cut_point]), **dict(list(parent2.items())[cut_point:])}
                child2 = {**dict(list(parent2.items())[:cut_point]), **dict(list(parent1.items())[cut_point:])}
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Mutasyon
            for child in (child1, child2):
                if random.random() < mutation_prob:
                    key = random.choice(list(param_space.keys()))
                    if key == "learning_rate":
                        child[key] = random.uniform(*param_space["learning_rate"])
                    elif key == "batch_size":
                        child[key] = random.choice(param_space["batch_size"])
                    elif key == "hidden_layers":
                        child[key] = random.randint(*param_space["hidden_layers"])
                    elif key == "neurons_per_layer":
                        child[key] = random.randint(*param_space["neurons_per_layer"])
                    elif key == "activation":
                        child[key] = random.choice(param_space["activation"])
                    elif key == "optimizer":
                        child[key] = random.choice(param_space["optimizer"])

            new_population.extend([child1, child2])

        population = new_population[:pop_size]
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    return best_individual, best_fitness, all_generations_fitness

# Guguk Kuşu Algoritması
def cuckoo_search(param_space, dataset, num_nests=10, max_generations=5, pa=0.5):
    input_shape = (100, 100, 3)
    num_classes = 141

    nests = [
        {
            "learning_rate": random.uniform(*param_space["learning_rate"]),
            "batch_size": random.choice(param_space["batch_size"]),
            "hidden_layers": random.randint(*param_space["hidden_layers"]),
            "neurons_per_layer": random.randint(*param_space["neurons_per_layer"]),
            "activation": random.choice(param_space["activation"]),
            "optimizer": random.choice(param_space["optimizer"])
        }
        for _ in range(num_nests)
    ]

    fitnesses = [fitness_function(nest, dataset, input_shape, num_classes) for nest in nests]
    best_nest = nests[np.argmax(fitnesses)]
    best_fitness = max(fitnesses)
    all_generations_fitness = []

    for generation in range(max_generations):
        for i in range(num_nests):
            new_nest = nests[i].copy()
            key = random.choice(list(param_space.keys()))
            if key == "learning_rate":
                new_nest[key] = random.uniform(*param_space["learning_rate"])
            elif key == "batch_size":
                new_nest[key] = random.choice(param_space["batch_size"])
            elif key == "hidden_layers":
                new_nest[key] = random.randint(*param_space["hidden_layers"])
            elif key == "neurons_per_layer":
                new_nest[key] = random.randint(*param_space["neurons_per_layer"])
            elif key == "activation":
                new_nest[key] = random.choice(param_space["activation"])
            elif key == "optimizer":
                new_nest[key] = random.choice(param_space["optimizer"])

            new_fitness = fitness_function(new_nest, dataset, input_shape, num_classes)
            if new_fitness > fitnesses[i]:
                nests[i] = new_nest
                fitnesses[i] = new_fitness

        if max(fitnesses) > best_fitness:
            best_fitness = max(fitnesses)
            best_nest = nests[np.argmax(fitnesses)]

        all_generations_fitness.append(best_fitness)  # Bu jenerasyonun en iyi fitness'ını kaydet

        for i in range(int(pa * num_nests)):
            nests[random.randint(0, num_nests - 1)] = {
                "learning_rate": random.uniform(*param_space["learning_rate"]),
                "batch_size": random.choice(param_space["batch_size"]),
                "hidden_layers": random.randint(*param_space["hidden_layers"]),
                "neurons_per_layer": random.randint(*param_space["neurons_per_layer"]),
                "activation": random.choice(param_space["activation"]),
                "optimizer": random.choice(param_space["optimizer"])
            }

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    return best_nest, best_fitness, all_generations_fitness

# Veri Yükleme
def load_fruits360_dataset():
    dataset_path = "C:/Users/DELL/Downloads/fruits-360/fruits-360_dataset_100x100/fruits-360"
    datagen = ImageDataGenerator(
        rescale=1.0/255.0, validation_split=0.2
    )

    train_data = datagen.flow_from_directory(
        dataset_path + '/Training',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    val_data = datagen.flow_from_directory(
        dataset_path + '/Training',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    return train_data, val_data

if __name__ == "__main__":
    dataset = load_fruits360_dataset()

    print("Running Genetic Algorithm for 10 generations...")
    ga_best_10, ga_fitness_10, ga_fitness_per_gen_10 = genetic_algorithm(param_space, dataset, generations=10)
    print("Best Individual:", ga_best_10)
    print("Best Fitness:", ga_fitness_10)

    print("\nRunning Genetic Algorithm for 20 generations...")
    ga_best_20, ga_fitness_20, ga_fitness_per_gen_20 = genetic_algorithm(param_space, dataset, generations=20)
    print("Best Individual:", ga_best_20)
    print("Best Fitness:", ga_fitness_20)

    print("\nRunning Cuckoo Search for 10 generations...")
    cs_best_10, cs_fitness_10, cs_fitness_per_gen_10 = cuckoo_search(param_space, dataset, max_generations=10)
    print("Best Individual:", cs_best_10)
    print("Best Fitness:", cs_fitness_10)

    print("\nRunning Cuckoo Search for 20 generations...")
    cs_best_20, cs_fitness_20, cs_fitness_per_gen_20 = cuckoo_search(param_space, dataset, max_generations=20)
    print("Best Individual:", cs_best_20)
    print("Best Fitness:", cs_fitness_20)

    print("\nGA Fitness per Generation (10 gens):", ga_fitness_per_gen_10)
    print("GA Fitness per Generation (20 gens):", ga_fitness_per_gen_20)
    print("CS Fitness per Generation (10 gens):", cs_fitness_per_gen_10)
    print("CS Fitness per Generation (20 gens):", cs_fitness_per_gen_20)


Found 56432 images belonging to 141 classes.
Found 14059 images belonging to 141 classes.
Running Genetic Algorithm for 10 generations...
Generation 1: Best Fitness = 0.8006259202957153
Generation 2: Best Fitness = 0.8006259202957153
Generation 3: Best Fitness = 0.8066718578338623
Generation 4: Best Fitness = 0.8224624991416931
Generation 5: Best Fitness = 0.8224624991416931
Generation 6: Best Fitness = 0.8416672348976135
Generation 7: Best Fitness = 0.8416672348976135
Generation 8: Best Fitness = 0.8465040326118469
Generation 9: Best Fitness = 0.8465040326118469
Generation 10: Best Fitness = 0.8465040326118469
Best Individual: {'learning_rate': 0.030169917297492074, 'batch_size': 32, 'hidden_layers': 5, 'neurons_per_layer': 65, 'activation': 'relu', 'optimizer': 'adam'}
Best Fitness: 0.8465040326118469

Running Genetic Algorithm for 20 generations...
Generation 1: Best Fitness = 0.7188989520072937
Generation 2: Best Fitness = 0.7538943290710449
Generation 3: Best Fitness = 0.753894329